##### 问题:
我们想让对象支持上下文管理协议（context-management protocol，通过with语句触发）。

##### 解决方案:
要让对象能够兼容with语句，需要实现__enter__()和__exit__()方法。比方说，考虑下面这个表示网络连接的类：

In [2]:
from socket import socket, AF_INET, SOCK_STREAM 
class LazyConnection: 
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None 
    def __enter__(self): 
        if self.sock is not None: 
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address) 
        return self.sock 
    def __exit__(self, exc_ty, exc_val, tb): 
        self.sock.close() 
        self.sock = None 


这个类的核心功能就是表示一条网络连接，但是实际上在初始状态下它并不会做任何事情（比如，它并不会建立一条连接）。相反，网络连接是通过with语句来建立和关闭的（这正是上下文管理的基本需求）。示例如下：

In [3]:
from functools import partial 
conn = LazyConnection(('www.python.org', 80)) # Connection closed
with conn as s:
# conn.__enter__() executes: connection open

    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))  
      
# conn.__exit__() executes: connection closed 

要编写一个上下文管理器，其背后的主要原则就是我们编写的代码需要包含在由with语句定义的代码块中。当遇到with语句时，\_\_enter\_\_()方法首先被触发执行。\_\_enter\_\_()的返回值（如果有的话）被放置在由as限定的变量当中。之后开始执行with代码块中的语句。最后，\_\_exit\_\_()方法被触发来执行清理工作。

这种形式的控制流与with语句块中发生了什么情况是没有关联的，出现异常时也是如此。实际上，\_\_exit\_\_()方法的三个参数就包含了异常类型、值和对挂起异常的追溯（如果出现异常的话）。\_\_exit\_\_()方法可以选择以某种方式来使用异常信息，或者什么也不干直接忽略它并返回None作为结果。如果\_\_exit\_\_()返回True，异常就会被清理干净，好像什么都没发生过一样，而程序也会立刻继续执行with语句块之后的代码。

这项技术有一个微妙的地方，那就是LazyConnection类是否可以通过多个with语句以嵌套的方式使用socket连接。正如我们给出的代码那样，一次只允许创建一条单独的socket连接。当socket已经在使用时，如果尝试重复使用with语句就会产生异常。我们可以对这个实现稍做修改来绕过这个限制，示例如下：

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM 
class LazyConnection: 
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM 
        self.connections = []
    def __enter__(self):
        sock = socket(self.family, self.type) 
        sock.connect(self.address) 
        self.connections.append(sock) 
        return sock 
    def __exit__(self, exc_ty, exc_val, tb): 
        self.connections.pop().close() 
# Example use
from functools import partial 
conn = LazyConnection(('www.python.org', 80)) 
with conn as s1:     
    pass
with conn as s2:
    pass

在第二个版本中，LazyConnection成了一个专门生产网络连接的工厂类。在内部实现中，我们把一个列表当成栈使用来保存连接。每当__enter__()执行时，由它产生一个新的连接并添加到栈中。而__exit__()方法只是简单地将最近加入的那个连接从栈中弹出并关闭它。这个修改很微不足道，但是这样就可以允许用嵌套式的with语句一次创建出多个连接了。

上下文管理器最常用在需要管理类似文件、网络连接和锁这样的资源的程序中。这些资源的关键点在于它们必须显式地进行关闭或释放才能正确工作。例如，如果获得了一个锁，之后就必须确保要释放它，否则就会有死锁的风险。通过实现__enter__()和__exit__()，并且利用with语句来触发，这类问题就可以很容易地避免了。因为__exit__()方法中的清理代码无论如何都会保证运行的。